In [53]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""

@author: bjang
"""

import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys

class eFollettScraperCalvin:
    def __init__(self, search_url = 'https://www.bkstr.com/calvinstore/shop/textbooks-and-course-materials',
                 headless = True):
        
        #set parameters
#        self.xpath = xpath
#        self.headerpath = headerpath
#        self.user = user
#        self.pw = pw

        # login with credentials
        chrome_options = webdriver.ChromeOptions()
        if headless is True:
            chrome_options.add_argument('--headless')
            chrome_options.add_argument('--window-size=1280x1696')
        
        #open search page
        self.driver = webdriver.Chrome(chrome_options=chrome_options)
        self.driver.get(search_url)
        time.sleep(4)
        
        print('You have logged in successfully.')
        

    def setDates(self, start_date, end_date):
        #set date(s), if given
        if start_date is not None and end_date is not None:
            #click "date" radio button
            self.driver.find_element_by_xpath('/html/body/div[4]/div[2]/section/div[2]/div[2]/report-generator/div/div/div[1]/div/div[1]/div[1]/div/div/div[2]/date-template/div/div[2]/div/div[2]/label[1]/span[1]').click()
            
            #enter start_date
            textinput = self.driver.find_element_by_xpath('/html/body/div[4]/div[2]/section/div[2]/div[2]/report-generator/div/div/div[1]/div/div[1]/div[1]/div/div/div[2]/date-template/div/div[2]/div/div[2]/label[1]/div/div/bootstrap-double-datepicker/div/input[1]')
            textinput.send_keys(Keys.CONTROL, 'a') #select all text
            textinput.send_keys(Keys.BACKSPACE) #delete all text
            textinput.send_keys(start_date)
            time.sleep(1)
            
            #enter end date
            textinput = self.driver.find_element_by_xpath('/html/body/div[4]/div[2]/section/div[2]/div[2]/report-generator/div/div/div[1]/div/div[1]/div[1]/div/div/div[2]/date-template/div/div[2]/div/div[2]/label[1]/div/div/bootstrap-double-datepicker/div/input[2]')
            textinput.send_keys(Keys.CONTROL, 'a') #select all text
            textinput.send_keys(Keys.BACKSPACE) #delete all text
            textinput.send_keys(end_date)
            time.sleep(1)
            
            #click "go" to load report
            self.driver.find_element_by_xpath('/html/body/div[4]/div[2]/section/div[2]/div[2]/report-generator/div/div/div[3]/div[3]/button').click()
            time.sleep(5)
    
    def scrape(self, CSVname = None, start_date = None, end_date = None):
        #get table and header labels
        data = self.driver.find_elements_by_xpath(self.xpath)   
        data_header = self.driver.find_elements_by_xpath(self.headerpath)
        data_labels = (data_header[0].text).split('\n')
        
        #save table in pandas df
        df = pd.DataFrame(columns=data_labels)
        for d in data:
            df.loc[len(df)] = (d.text).split('\n')
            
        #export df to csv
        if CSVname is not None:
            df.to_csv(CSVname)
            print('CSV file logged')
        
        return df
        
    def close(self):
        #close out driver
        self.driver.close()
        

In [54]:
scraper = eFollettScraperCalvin(headless=False)

/Users/cj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: use options instead of chrome_options


You have logged in successfully.


In [55]:
# Click shop by course tab
shopbycourse = scraper.driver.find_element_by_xpath('//*[@id="content"]/div/label[1]')
shopbycourse.click()

In [56]:
# Department
depelement = scraper.driver.find_element_by_xpath('//*[@id="departmentIdSelect"]')
departments = []
for depoption in depelement.find_elements_by_tag_name('option'):
    departments.append(depoption.text)
    #departments.append(option.get_attribute('value'))

In [34]:
# Courses
crselement = scraper.driver.find_element_by_xpath('//*[@id="courseIdSelect"]')
courses = []
for crsoption in crselement.find_elements_by_tag_name('option'):
    courses.append(crsoption.text)

In [37]:
# Sections
sctelement = scraper.driver.find_element_by_xpath('//*[@id="sectionIdSelect"]')
sections = []
for secoption in sctelement.find_elements_by_tag_name('option'):
    sections.append(secoption.text)

In [57]:
# Department
depelement = scraper.driver.find_element_by_xpath('//*[@id="departmentIdSelect"]')
data_labels = ["Department", "Course", "Section"]
mastercourses = pd.DataFrame(columns=data_labels)

for depoption in depelement.find_elements_by_tag_name('option'):
    depoption.click()
    
    # Course
    crselement = scraper.driver.find_element_by_xpath('//*[@id="courseIdSelect"]')
    for crsoption in crselement.find_elements_by_tag_name('option'):
        crsoption.click()
        
        # Section
        sctelement = scraper.driver.find_element_by_xpath('//*[@id="sectionIdSelect"]')
        for sctoption in sctelement.find_elements_by_tag_name('option'):
            sctoption.click()
            #print(depoption.text,crsoption.text,)
            if sctoption != "Select Your Section":
                #print(depoption.text,crsoption.text,sctoption.text)
                mastercourses.loc[len(mastercourses)] = [depoption.text,crsoption.text,sctoption.text]

            
    
    


In [58]:
mastercourses

,Department,Course,Section
0,ACCT,203,Select Your Section
1,ACCT,203,B
2,ACCT,203,C
3,ACCT,203,D
4,ACCT,203,E
5,ACCT,204,Select Your Section
6,ACCT,204,A
7,ACCT,204,B
8,ACCT,205,Select Your Section
9,ACCT,205,A


In [59]:
mastercourses.to_csv('courses.csv', index=False)
